In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import json

In [2]:
def read_graph(file):
    with open(file, "r") as f:
        first_line = f.readline().strip().split()
        vnf_types = int(first_line[0])
        second_line = int(f.readline().strip())
        list_servers = {}
        list_swtiches = {}
        for _ in range(second_line):
            line = f.readline().strip().split()
            description = {}
            if int(line[2]) != -1:
                description["delay"] = int(line[1])
                description["active_cost"] = int(line[2])
                list_cosst = [int(line[3 + j]) for j in range(vnf_types)]
                description["list_install_cost"] = list_cosst
                list_servers[int(line[0])] = description
            else:
                description["delay"] = int(line[1])
                list_swtiches[int(line[0])] = description
        number_edges = int(f.readline().strip())
        edges = {(int(line[0]), int(line[1])): int(line[2]) for line in [f.readline().strip().split() for _ in range(number_edges)]}
        list_vnfs = list(range(vnf_types))
    return list_servers, list_swtiches, edges, list_vnfs

In [3]:
server_detail ,switch_detail ,edge_detail, VNFs = read_graph("input.txt")

In [6]:
def read_sfc(filepath):
    try:
        with open(filepath, "r") as f:
            nums_request = int(f.readline().strip())
            list_request = []
            for _ in range(nums_request):
                line = f.readline().strip().split()
                description = {
                    'memory': int(line[0]),
                    'bandwidth': int(line[1]),
                    'cpu': int(line[2]),
                    'source': int(line[3]),
                    'destination': int(line[4]),
                    'vnfs_chain': [int(v) for v in line[5:]]
                }
                list_request.append(description)
        with open("request.json", "w") as outfile:
            json.dump(list_request, outfile, indent=4)
        return list_request
    except Exception as e:
        print(e)
        return None

In [ ]:
request_detail = read_sfc("request10.txt")

In [16]:
server_detail, switch_detail, edge_detail, VNFs = read_graph("input.txt")
request_detail = read_sfc("request10.txt")
R = list(range(len(request_detail)))
V = list(server_detail.keys())
S = list(switch_detail.keys())
E = list(edge_detail.keys())

In [14]:
model = ConcreteModel()

In [ ]:
model.V = Set(initialize=V)
model.S = Set(initialize=S)
model.E = Set(initialize=E)
model.VNFs = Set(initialize=VNFs)
model.R = Set(initialize=R)

In [19]:
# tạo các biến quyết định
# x_v = 1 nếu v được kích hoạt, 0 nếu không
model.x = Var(model.V,  within=Binary)
# Biến x_{vk}^j: VNF f_k được phục vụ trong máy chủ v cho dịch vụ j
model.x_j_vk = Var(model.V, model.VNFs, model.R, within=Binary)
# Biến x'_{vk}: f_k được cài đặt trên máy chủ v cho bất kỳ dịch vụ nào
model.x_prime_vk = Var(model.V, model.VNFs, within=Binary)
# Biến y_{vu}^j: SFC r_j đi qua liên kết (v, u) hoặc (u, v)
model.y_j_vu = Var(model.E, model.R, within=Binary)

In [20]:
#tính bandwidth tối đa có thể của mỗi cạnh
def max_bandwidth(request_detal):
    max_bandwith = 0
    for i in range(len(request_detal)):
        max_bandwith += request_detal[i]['bandwidth']*(len(request_detal[i]['vnfs_chain'])+1) 
    return max_bandwith


#tính memory tối đa có thể của mỗi switch_node
def max_memory(request_detal):
    max_memory = 0
    for i in range(len(request_detal)):
        max_memory += request_detal[i]['memory']*(len(request_detal[i]['vnfs_chain'])+1)
    return max_memory


#tính cpu tối đa có thể của mối server_node
def max_cpu(request_detal):
    max_cpu = 0
    for i in range(len(request_detal)):
        max_cpu += request_detal[i]['cpu']*(len(request_detal[i]['vnfs_chain']))
    return max_cpu

In [21]:
bandwidth_limit = max_bandwidth(request_detail)
memory_limit = max_memory(request_detail)
cpu_limit = max_cpu(request_detail)

In [ ]:
def minimum_delay(E,V,R,VNF,request_detal,y_vu_j,x_j_vk,server_detail,edge_detail):
    edge_delay = 0
    server_delay = 0
    for index in R:
        for (u,v) in E:
            edge_delay += y_vu_j[u,v,index]*edge_detail[(u,v)]
        for v in V:
            for f in VNF:
                if f in request_detal[index]['vnfs_chain']:
                    server_delay+=x_j_vk[v,f,index] * server_detail[v]['delay']
    current_delay = edge_delay+server_delay
    max_delay = 0
    for i in edge_detail.values():
        max_delay+=i
    for s in server_detail.keys():
        max_delay+=server_detail[s]['delay']
    max_delay*=len(R)
    return current_delay/max_delay

def minimum_cost_active(V,server_detail,x):
    cost_active = 0
    for i in V:
        cost_active+=x[i]*server_detail[i]['active_cost']
    max_cost = sum(server_detail[i]['active_cost'] for i in server_detail)
    return cost_active/max_cost

def minimum_cost_install_vnf(V,VNF,x,x_prime_vk,server_detail):
    cost_install  = 0
    for i in V:
        for fk in VNF:
            cost_install+=x[i]*x_prime_vk[i,fk] * server_detail[i]['list_install_cost'][fk]
    max_cost_install = 0
    for i in server_detail.keys():
        for j in server_detail[i]['list_install_cost']:
            max_cost_install+=j
    return cost_install/max_cost_install

In [ ]:
def bandwidth_rule(model):
    return (sum(model.y_j_vu[e,r]*request_detail[r]['bandwidth'] for e in model.E for r in model.R) <= bandwidth_limit)
def memory_rule(model):
    return (sum(model.x[v]*server_detail[v]['memory'] for v in model.V) <= memory_limit)

In [ ]:
def objective_function(model):
    return minimum_delay(model.E,model.V,model.R,model.VNFs,request_detail,model.y_j_vu,model.x_j_vk,server_detail,edge_detail) + minimum_cost_active(model.V,server_detail,model.x) + minimum_cost_install_vnf(model.V,model.VNFs,model.x,model.x_prime_vk,server_detail)
model.objective = Objective(rule=objective_function, sense=minimize)
